In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd "/content/gdrive/My Drive/aods"

/content/gdrive/My Drive/aods


In [4]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [39.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubu

In [5]:
! pip install pytube3
! pip install selenium
! pip install face_recognition

     |████████████████████████████████| 911kB 3.3MB/s 
     |████████████████████████████████| 100.2MB 110kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=5c0cab2bc183c89c4f72f66dffc7944a7c2dc4f3ae065393fb4ca6f4e27fe905
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [7]:
import urllib
from bs4 import BeautifulSoup
from pytube import YouTube
import cv2
import os
import glob
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
import sys
import cv2
import dlib
import face_recognition


In [8]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)
# wd.get("https://www.webite-url.com")

In [18]:
def get_urls(text, limit=5):
    '''Search for a phrase on Youtube and returns a list of links to the first videos
        that are returned. A maximum of results can be set (default 10).'''

    query = urllib.parse.quote(text)
    url = "https://www.youtube.com/results?search_query=" + query
    driver.get(url)
    # l = driver.find_elements_by_class_name("style-scope.ytd-item-section-renderer")
    # h = l[0].find_element_by_class_name("yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail")
    video_url_raw_list = driver.find_elements_by_class_name("yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail")

    urls = []
    for i, vid_url in enumerate(video_url_raw_list):
        if i < limit:
            urls.append(vid_url.get_attribute('href'))
    print(f"Found {len(urls)} video links for {text}")

    # response = urllib.request.urlopen(url)
    # html = response.read()
    # soup = BeautifulSoup(html, 'html.parser')
    # urls = []
    # for i, vid in enumerate(soup.findAll(attrs={'class': 'yt-uix-tile-link'})):
    #     if i < limit:
    #         urls.append('https://www.youtube.com' + vid['href'])
    # print(f"Found {len(urls)} video links for {text}")
    return urls


def download_video(url, path="/content/gdrive/My Drive/aods/videos", max_duration=10):
    '''Download Youtube video from the url to the path specified, or the cwd if non specified.
        Only videos shorter than max_duration are downloaded and reports are printed to say which
         video is downloaded successfully.'''

    try:
        yt = YouTube(url)
        # duration = int(yt.player_config_args['player_response']['streamingData']['formats'][0]['approxDurationMs'])
        # if duration < max_duration * 60 * 1000:
        yt = yt.streams.filter(file_extension='mp4').first()
        print("Downloading video...")
        out_file = yt.download(path)
        file_name = out_file.split("\\")[-1]
        print(f"Downloaded {file_name} correctly!")
        # else:
        #     print(f"Video {url} too long")
    except Exception as exc:
        print(f"Download of {url} did not work because of {exc}...")


def max_label(name, folder):
    '''Look at a folder and check the files with pattern "name_###.jpg" to extract the
    largest label present.'''

    path_pattern = os.path.join(folder, "*.jpg")
    existing_files = glob.glob(path_pattern)
    if not existing_files:
        biggest_label = 0
    else:
        existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)
        biggest_label = max(existing_labels)
    return biggest_label


def extract_images_from_video(video, folder="/content/gdrive/My Drive/aods/images", delay=30, name="video_image", max_images=20, silent=False):
    '''Read a downloaded video from its path and extract screenshots every few seconds, set by the delay parameter.
        Images are saved in the specified folder or the cwd if none is specified and a maximum number of
        screenshots can be specified. The files are named "name_##.jpg" and the labelling starts where it already stops
        in the folder.'''

    vidcap = cv2.VideoCapture(video)
    count = 0
    num_images = 0
    if not folder:
        folder = os.getcwd()
    label = max_label(name, folder)
    success = True
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))

    while success and num_images < max_images:
        success, image = vidcap.read()
        num_images += 1
        label += 1
        file_name = name + "_" + str(label) + ".jpg"
        path = os.path.join(folder, file_name)
        try:
          cv2.imwrite(path, image)
          if cv2.imread(path) is None:
              os.remove(path)
          else:
              if not silent:
                  print(f'Image successfully written at {path}')
        except Exception as exe:
          # print("ERROR!!!", exe)
          pass
        count += delay * fps
        vidcap.set(1, count)

def extract_images_from_video_using_face_recog(video, folder="/content/gdrive/My Drive/aods/images", delay=30, name="video_image", max_images=50, silent=False):
    '''Read a downloaded video from its path and extract screenshots every few seconds, set by the delay parameter.
        Images are saved in the specified folder or the cwd if none is specified and a maximum number of
        screenshots can be specified. The files are named "name_##.jpg" and the labelling starts where it already stops
        in the folder.'''


    video_capture = cv2.VideoCapture(video)

    count = 0
    num_images = 0
    if not folder:
      folder = os.getcwd()
    label = max_label(name, folder)
    success = True
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))


    while success and num_images < max_images:
    # success, image = vidcap.read()

    # Grab a single frame of video
      success, frame = video_capture.read()
      # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
      try:
        rgb_frame = frame[:, :, ::-1]
      except Exception as exe:
        # print("Error!!! ", exe)
        pass
      num_images += 1
      label += 1
      file_name = name + "_" + str(label) + ".jpg"
      path = os.path.join(folder, file_name)

      print("Processing frame [{}/{}]".format(count, number_of_frames))
      # print("finding the face")
      # Find all the faces in the current frame of video
      face_locations = face_recognition.face_locations(rgb_frame)

      # print("Writing a face image...")
      # Display the results
      extra = 100

      for top, right, bottom, left in face_locations:
          # Draw a box around the face
          try:
            top = top - extra
            bottom = bottom + extra
            left = left - extra
            right = right + extra
            # cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            crop_img = frame[top:bottom, left:right]
            cv2.imwrite(path, crop_img)

            if cv2.imread(path) is None:
                os.remove(path)
            else:
                if not silent:
                    print(f'Image successfully written at {path}')
          except Exception as exe:
            # print("ERROR!!!", exe)
            pass

      count += delay * fps
      video_capture.set(1, count)


def extract_images_from_word(text, delete_video=False, image_delay=30,
                             num_urls=10, max_images=50, name="video_image", max_duration=15, silent=False, folder_name = "/content/gdrive/My Drive/aods/images"):
    '''Search for a phrase on Youtube, download a specified number of videos from the results and extract
        screenshots from them with a specified time delay between each. A folder is created in the cwd
         to store the images, which are named "name_###.jpg". If the folder already exists, the labelling
         starts where it stopped before. Videos are deleted after the extraction.'''

    if not os.path.exists(name):
        os.mkdir(name)
    urls = get_urls(text, num_urls)
    for url in urls:
        download_video(url, max_duration=max_duration)
    
    videos = glob.glob("/content/gdrive/My Drive/aods/videos/*.mp4")
    for i, video in enumerate(videos):
        extract_images_from_video(video, folder=folder_name, delay=image_delay, name=name, max_images=max_images,
                                  silent=silent)
        # extract_images_from_video_using_face_recog(video, folder=folder_name, delay=image_delay, name=name, max_images=max_images,
        #                           silent=silent)
        if delete_video:
            os.remove(video)



In [19]:
extract_images_from_word("Friends and big bang theory", delete_video=True)

Found 8 video links for Friends and big bang theory
Downloaded /content/gdrive/My Drive/aods/videos/10 Scenes From The Big Bang Theory Which Are Copied From FRIENDS -  Part I.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/The Big Bang Theory vs FRIENDS.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/The Big Bang Theory Cast Interview - Tribute To Director James Burrows.mp4 correctly!
Download of https://www.youtube.com/watch?v=eM9m09uDYFk did not work because of 'cipher'...
Downloaded /content/gdrive/My Drive/aods/videos/Kaley Cuoco Freaks Out Over Friends Reunion See Her Pic With the Cast!.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/The Big Bang Theory- Sheldon making friends (The Scientific Way).mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Rachel Green Vs Penny Hofstadter ( Friends Vs Big Bang Theory ).mp4 correctly!
Download of None did not work because of expected string or bytes-like object...
Image successfully w

In [20]:
extract_images_from_word("Leonardo DiCaprio winning Best Actor", delete_video=False)



Found 10 video links for Leonardo DiCaprio winning Best Actor
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprio winning Best Actor.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprio wins Best Leading Actor award - The British Academy Film Awards 2016 - BBC One.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprio Won Best Actor Drama Golden Globe 2016.mp4 correctly!
Download of https://www.youtube.com/watch?v=AOoP56eXtzM did not work because of 'cipher'...
Downloaded /content/gdrive/My Drive/aods/videos/Golden Globes 2005 Leonardo DiCaprio Best Actor Drama.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprio Best Acting Scenes.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprio Wins Male Actor Leading Role SAG Awards 2016.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Leonardo DiCaprios Reaction to Lady Gagas Golden Globes Win is Absolutely P

In [21]:
extract_images_from_word("The Funniest Friends Moments of Season 6", delete_video=False)



Found 10 video links for The Funniest Friends Moments of Season 6
Download of https://www.youtube.com/watch?v=penAGgSqaNE did not work because of 'cipher'...
Download of https://www.youtube.com/watch?v=WSND07qqzgk did not work because of get_ytplayer_config: could not find match for config_patterns...
Downloaded /content/gdrive/My Drive/aods/videos/The Funniest Friends Moments of Season 7 (4 - 1).mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/Friends  Season 06 - Top 50 Best Scenes.mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/The Funniest Friends Moments of Season 7 (4 - 2).mp4 correctly!
Downloaded /content/gdrive/My Drive/aods/videos/The Funniest Friends Moments of Season 8 (5 - 1).mp4 correctly!
Download of https://www.youtube.com/watch?v=o7PZ7yNLHzk did not work because of 'cipher'...
Downloaded /content/gdrive/My Drive/aods/videos/The Funniest Friends Moments of Season 5 (3 - 1).mp4 correctly!
Download of https://www.youtube.com/watch?v=9Vl72Ct3Ne

In [ ]:
!zip -r /content/gdrive/My Drive/aods/images.zip /content/gdrive/My Drive/aods/rand

from google.colab import files
files.download("/content/random_image.zip")

In [9]:
videos = glob.glob("/content/gdrive/My Drive/aods/videos/*.mp4")

In [10]:
video = videos[0]
video

'/content/gdrive/My Drive/aods/videos/tbbt.mp4'

In [ ]:
# def extract_images_from_video(video, folder="/content/gdrive/My Drive/aods/images", delay=30, name="file", max_images=20, silent=False):
#     '''Read a downloaded video from its path and extract screenshots every few seconds, set by the delay parameter.
#         Images are saved in the specified folder or the cwd if none is specified and a maximum number of
#         screenshots can be specified. The files are named "name_##.jpg" and the labelling starts where it already stops
#         in the folder.'''

# video = videos[0]
import time
video = '/content/gdrive/My Drive/aods/videos/tbbt.mp4'
folder="/content/gdrive/My Drive/aods/images"
delay=30
name="video_image"
max_images=900
silent=False

# vidcap = cv2.VideoCapture(video)
video_capture = cv2.VideoCapture(video)

count = 0
num_images = 0
if not folder:
    folder = os.getcwd()
label = max_label(name, folder)
success = True
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
number_of_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))


while success and num_images < max_images:
    # success, image = vidcap.read()

    # Grab a single frame of video
    success, frame = video_capture.read()
    time.sleep(1)
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]
    
    num_images += 1
    label += 1
    file_name = name + "_" + str(label) + ".jpg"
    path = os.path.join(folder, file_name)

    print("Processing frame [{}/{}]".format(count, number_of_frames))
    # print("finding the face")
    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)

    # print("Writing a face image...")
    # Display the results
    extra = 100

    for top, right, bottom, left in face_locations:
        # Draw a box around the face
        try:
          top = top - extra
          bottom = bottom + extra
          left = left - extra
          right = right + extra
          cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
          crop_img = frame[top:bottom, left:right]
          cv2.imwrite(path, crop_img)

          if cv2.imread(path) is None:
              os.remove(path)
          else:
              if not silent:
                  print(f'Image successfully written at {path}')
        except:
          print("Error...")
    count += delay * fps
    video_capture.set(1, count)


Processing frame [0/915]
Image successfully written at /content/gdrive/My Drive/aods/images/video_image_71.jpg
Image successfully written at /content/gdrive/My Drive/aods/images/video_image_71.jpg
Processing frame [870/915]
Image successfully written at /content/gdrive/My Drive/aods/images/video_image_72.jpg
Image successfully written at /content/gdrive/My Drive/aods/images/video_image_72.jpg


TypeError: ignored

In [ ]:
for i, video in enumerate(glob.glob("/content/gdrive/My Drive/aods/videos/*.mp4")):
        extract_images_from_video(video, folder=name, delay=image_delay, name=name, max_images=max_images,
                                  silent=silent)
        if delete_video:
            os.remove(video)

In [ ]:
def max_label(name, folder):
    '''Look at a folder and check the files with pattern "name_###.jpg" to extract the
    largest label present.'''

    path_pattern = os.path.join(folder, "*.jpg")
    existing_files = glob.glob(path_pattern)
    if not existing_files:
        biggest_label = 0
    else:
        existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)
        biggest_label = max(existing_labels)
    return biggest_label

In [ ]:
label = max_label("video_image", "/content/gdrive/My Drive/aods/images")

In [ ]:
name = "video_image"
folder = "/content/gdrive/My Drive/aods/images"

In [ ]:
path_pattern = os.path.join(folder, "*.jpg")
path_pattern

'/content/gdrive/My Drive/aods/images/*.jpg'

In [ ]:
existing_files = glob.glob(path_pattern)
existing_files

['/content/gdrive/My Drive/aods/images/file_1.jpg',
 '/content/gdrive/My Drive/aods/images/file_2.jpg',
 '/content/gdrive/My Drive/aods/images/file_3.jpg',
 '/content/gdrive/My Drive/aods/images/file_4.jpg',
 '/content/gdrive/My Drive/aods/images/file_5.jpg',
 '/content/gdrive/My Drive/aods/images/file_6.jpg',
 '/content/gdrive/My Drive/aods/images/file_7.jpg',
 '/content/gdrive/My Drive/aods/images/file_8.jpg',
 '/content/gdrive/My Drive/aods/images/file_9.jpg',
 '/content/gdrive/My Drive/aods/images/file_10.jpg',
 '/content/gdrive/My Drive/aods/images/file_11.jpg',
 '/content/gdrive/My Drive/aods/images/file_12.jpg',
 '/content/gdrive/My Drive/aods/images/file_13.jpg',
 '/content/gdrive/My Drive/aods/images/file_14.jpg',
 '/content/gdrive/My Drive/aods/images/file_15.jpg',
 '/content/gdrive/My Drive/aods/images/file_16.jpg',
 '/content/gdrive/My Drive/aods/images/file_17.jpg',
 '/content/gdrive/My Drive/aods/images/file_18.jpg',
 '/content/gdrive/My Drive/aods/images/file_19.jpg',
 '

In [ ]:
existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)

In [ ]:
max(existing_labels)

20

In [ ]:
path_pattern = os.path.join(folder, name + "_*.jpg")
existing_files = glob.glob(path_pattern)
if not existing_files:
    biggest_label = 0
else:
    existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)
    biggest_label = max(existing_labels)

In [13]:
!nvidia-smi


Mon Oct 12 08:36:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    32W / 250W |    307MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
# def extract_images_from_video(video, folder="/content/gdrive/My Drive/aods/images", delay=30, name="video_image", max_images=20, silent=False):
#     '''Read a downloaded video from its path and extract screenshots every few seconds, set by the delay parameter.
#         Images are saved in the specified folder or the cwd if none is specified and a maximum number of
#         screenshots can be specified. The files are named "name_##.jpg" and the labelling starts where it already stops
#         in the folder.'''

video = '/content/gdrive/My Drive/aods/videos/tbbt.mp4'
folder="/content/gdrive/My Drive/aods/images"
delay=30
name="video_image"
max_images=50
silent=False

vidcap = cv2.VideoCapture(video)
count = 0
num_images = 0
if not folder:
    folder = os.getcwd()
label = max_label(name, folder)
success = True
fps = int(vidcap.get(cv2.CAP_PROP_FPS))

while success and num_images < max_images:
    success, image = vidcap.read()
    # print(len(image))
    num_images += 1
    label += 1
    file_name = name + "_" + str(label) + ".jpg"
    path = os.path.join(folder, file_name)
    cv2.imwrite(path, image)
    if cv2.imread(path) is None:
        os.remove(path)
    else:
        if not silent:
            print(f'Image successfully written at {path}')
    count += delay * fps
    vidcap.set(1, count)


Image successfully written at /content/gdrive/My Drive/aods/images/video_image_9.jpg
Image successfully written at /content/gdrive/My Drive/aods/images/video_image_10.jpg


error: ignored